
# 🎛️ Clustering con K-Means — Versión Explicativa (v2)  
**Curso de Análisis de Datos con IA** · **Autores:** Doribel Rodríguez y Antonio Vantaggiato

**Objetivo pedagógico.** Comprender el algoritmo **K-Means** (objetivo, inercia, centroides y escalado), aplicar el **método del codo** y **silhouette**, y comparar un **dataset clásico sintético** con un **caso aplicado educativo/social**.



## 🧠 Conceptos esenciales

- **Clustering (agrupamiento).** Descubre estructuras en datos **sin etiquetas**.  
- **K-Means.** Asigna cada punto al **centroide** más cercano y actualiza los centroides como la media del cluster.  
- **Función objetivo (inercia).** Suma de distancias cuadráticas punto–centroide; el algoritmo intenta **minimizarla**.  
- **Escalado.** Variables en diferentes escalas distorsionan las distancias; usa `StandardScaler`.  
- **Elección de k.** Método del **codo** (cambios en inercia) y **coeficiente silhouette** (−1 a 1).


## 1) Preparación del entorno

In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
from sklearn.datasets import make_blobs

%matplotlib inline



## 2) Dataset clásico sintético: **make_blobs** (3 clústeres)


In [ ]:

X, y_true = make_blobs(n_samples=400, centers=3, cluster_std=1.1, random_state=42)
scaler = StandardScaler()
Xs = scaler.fit_transform(X)

inertias = []
silhouettes = []
ks = range(2, 8)
for k in ks:
    km = KMeans(n_clusters=k, n_init=10, random_state=42)
    km.fit(Xs)
    inertias.append(km.inertia_)
    silhouettes.append(silhouette_score(Xs, km.labels_))

fig, ax = plt.subplots(1,2, figsize=(11,4))
ax[0].plot(list(ks), inertias, marker="o")
ax[0].set_title("Método del codo (inercia)")
ax[0].set_xlabel("k")
ax[0].set_ylabel("Inercia")

ax[1].plot(list(ks), silhouettes, marker="o")
ax[1].set_title("Coeficiente silhouette")
ax[1].set_xlabel("k")
ax[1].set_ylabel("Silhouette")
plt.show()

# Elegimos k=3 y visualizamos
k_best = 3
km = KMeans(n_clusters=k_best, n_init=10, random_state=42).fit(Xs)
labels = km.labels_
centroids = km.cluster_centers_

plt.figure(figsize=(6,5))
plt.scatter(Xs[:,0], Xs[:,1], c=labels, s=40, alpha=0.8)
plt.scatter(centroids[:,0], centroids[:,1], marker="X", s=200, edgecolor="k")
plt.title("K-Means (k=3) — Datos escalados")
plt.xlabel("Componente 1 (estandarizada)")
plt.ylabel("Componente 2 (estandarizada)")
plt.grid(True)
plt.show()



## 3) Caso aplicado educativo/social: **hábitos estudiantiles** (dataset simulado realista)

Variables (simuladas):  
- **horas_estudio**, **asistencia_%**, **horas_pantalla/día**, **horas_sueño**, **promedio_prev**.  
Buscamos **perfiles** de estudiantes (p. ej., "alta dedicación", "equilibrados", "riesgo").


In [ ]:

rng = np.random.default_rng(1)
n = 240
horas_estudio = rng.normal(6, 2, n).clip(0, 12)
asistencia = rng.normal(82, 10, n).clip(40, 100)
horas_pantalla = rng.normal(4, 1.5, n).clip(0, 10)
horas_sueno = rng.normal(7, 1, n).clip(3, 10)
prom_prev = rng.normal(75, 12, n).clip(0, 100)

df = pd.DataFrame({
    "horas_estudio": horas_estudio.round(1),
    "asistencia_%": asistencia.round(1),
    "horas_pantalla": horas_pantalla.round(1),
    "horas_sueno": horas_sueno.round(1),
    "promedio_prev": prom_prev.round(1)
})
df.head()


In [ ]:

# Escalado y elección de k por codo y silhouette
sc = StandardScaler()
Xs2 = sc.fit_transform(df.values)

inertias2, silhouettes2, ks2 = [], [], range(2, 9)
for k in ks2:
    km2 = KMeans(n_clusters=k, n_init=10, random_state=42).fit(Xs2)
    inertias2.append(km2.inertia_)
    silhouettes2.append(silhouette_score(Xs2, km2.labels_))

fig, ax = plt.subplots(1,2, figsize=(11,4))
ax[0].plot(list(ks2), inertias2, marker="o")
ax[0].set_title("Codo — hábitos estudiantiles")
ax[0].set_xlabel("k"); ax[0].set_ylabel("Inercia")

ax[1].plot(list(ks2), silhouettes2, marker="o")
ax[1].set_title("Silhouette — hábitos estudiantiles")
ax[1].set_xlabel("k"); ax[1].set_ylabel("Silhouette")
plt.show()

k_sel = 3
km2 = KMeans(n_clusters=k_sel, n_init=10, random_state=42).fit(Xs2)
labels2 = km2.labels_
df_clusters = df.copy()
df_clusters["cluster"] = labels2
df_clusters.head()


### 3.1) Perfil de clústeres (medias por grupo)

In [ ]:

summary = df_clusters.groupby("cluster").mean().round(2)
display(summary)

# Visualización comparativa de medias por cluster (barras)
fig, axes = plt.subplots(1, 3, figsize=(14,4), sharey=True)
for c in range(3):
    means = summary.loc[c]
    axes[c].bar(means.index, means.values)
    axes[c].set_title(f"Cluster {c}")
    axes[c].tick_params(axis='x', rotation=45)
    axes[c].grid(axis="y", linestyle="--", alpha=0.5)
plt.suptitle("Medias por cluster — hábitos estudiantiles")
plt.tight_layout()
plt.show()



## 4) Cierre
- **K-Means** requiere **especificar k** y es sensible a la escala: usa estandarización.  
- Usa **codo** y **silhouette** como guía; valida la **interpretabilidad** de los grupos con el contexto.  
- En el caso aplicado, los clusters pueden sugerir perfiles de dedicación/riesgo; evita juicios deterministas y considera factores éticos.
